# Northern Brooklyn KSI Study

#### Output should be table with all the streets within region and KSI statistics in that region


####### Steps: 

####### 1. Convert the shapefile into a table with geometry
####### 2. Spatial join the shapefile with nysdot_all injury data
####### 3. Retrieve table with injury severity statistics that overlap with selected region


In [91]:
from ris import db2
from ris import pg_import_export_shps as shp
from IPython.display import clear_output
import datetime
from functools import reduce
import pandas as pd
from pandas import DataFrame
from pandas import ExcelWriter
import numpy as np
import os


clear_output()
timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
print 'Notebook run: {}'.format(timestamp)

# %load_ext sql 

Notebook run: 2019-06-12 18:43


In [ ]:
#db = db2.PostgresDb('dotdevpgsql02', 'GISGRID', quiet = True)
db = db2.PostgresDb('dotdevpgsql02', 'CRASHDATA', quiet = True)
#exporting study area to pg database
shp.import_shp_to_pg('C:\Users\soge\Desktop\Jupyter\Shps\Northern_Brooklyn_Study_Area\Study_Area.shp', db, schema='working', precision=False, 
                     permission=True, gdal_data=r"C:\Program Files (x86)\GDAL\gdal-data") 


User name (CRASHDATA):soge
Password (CRASHDATA)········


In [80]:

pgdb = db2.PostgresDb('dotdevpgsql02', 'CRASHDATA', quiet = True) #data connection
qry = (db2.query_to_table(pgdb, """
                                select inj as severity, count(inj) as cnt
                                from (
                                SELECT b.ext_of_inj inj, b.geom
                                FROM working.study_area a 
                                join public.nysdot_all b
                                on ST_DWithin(a.wkb_geometry, b.geom, 2)
                                where ext_of_inj is not null
                                and (ext_of_inj) like '%A%'
                                ) c
                                group by inj

                            """)
              )
print(qry)


User name (CRASHDATA):soge
Password (CRASHDATA)········
   severity  cnt
0         A  535
1        AA   21
2       AAA    5
3     AAAAA    1
4     AAACC    2
5       AAB    1
6      AABC    1
7       AAC    2
8        AB    7
9       ABB    3
10      ABC    3
11    ABCCC    2
12       AC   29
13      ACA    1
14     ACBB    1
15     ACBC    1
16      ACC    6
17     ACCC    6
18    ACXCB    1
19     ACXX    1
20       AK    1
21       BA    1
22     BBBA    1
23       CA    4
24     CAAA    1
25      CAB    1
26    CABCC    2
27      CAC    2
28     CACC    2
29    CACCC    1
30      CCA    1
31     CCAC    1
32       KA    1
33     KABA    1


In [81]:
for i,j in zip(qry.severity,qry.cnt):
    print(i,j)

('A', 535)
('AA', 21)
('AAA', 5)
('AAAAA', 1)
('AAACC', 2)
('AAB', 1)
('AABC', 1)
('AAC', 2)
('AB', 7)
('ABB', 3)
('ABC', 3)
('ABCCC', 2)
('AC', 29)
('ACA', 1)
('ACBB', 1)
('ACBC', 1)
('ACC', 6)
('ACCC', 6)
('ACXCB', 1)
('ACXX', 1)
('AK', 1)
('BA', 1)
('BBBA', 1)
('CA', 4)
('CAAA', 1)
('CAB', 1)
('CABCC', 2)
('CAC', 2)
('CACC', 2)
('CACCC', 1)
('CCA', 1)
('CCAC', 1)
('KA', 1)
('KABA', 1)


In [82]:
for i in qry.severity:
    print i
    


A
AA
AAA
AAAAA
AAACC
AAB
AABC
AAC
AB
ABB
ABC
ABCCC
AC
ACA
ACBB
ACBC
ACC
ACCC
ACXCB
ACXX
AK
BA
BBBA
CA
CAAA
CAB
CABCC
CAC
CACC
CACCC
CCA
CCAC
KA
KABA


In [83]:
count=0
for i in qry.severity:
    if 'A' in i:
        print i, i.count('A')
        count+=1
print(count)

A 1
AA 2
AAA 3
AAAAA 5
AAACC 3
AAB 2
AABC 2
AAC 2
AB 1
ABB 1
ABC 1
ABCCC 1
AC 1
ACA 2
ACBB 1
ACBC 1
ACC 1
ACCC 1
ACXCB 1
ACXX 1
AK 1
BA 1
BBBA 1
CA 1
CAAA 3
CAB 1
CABCC 1
CAC 1
CACC 1
CACCC 1
CCA 1
CCAC 1
KA 1
KABA 2
34


In [84]:
for i in qry.cnt:
    print i

535
21
5
1
2
1
1
2
7
3
3
2
29
1
1
1
6
6
1
1
1
1
1
4
1
1
2
2
2
1
1
1
1
1


In [85]:
for i,j in zip(qry.severity, qry.cnt):
    if 'A' in i:
        print(i,j,j*i.count('A'))

('A', 535, 535)
('AA', 21, 42)
('AAA', 5, 15)
('AAAAA', 1, 5)
('AAACC', 2, 6)
('AAB', 1, 2)
('AABC', 1, 2)
('AAC', 2, 4)
('AB', 7, 7)
('ABB', 3, 3)
('ABC', 3, 3)
('ABCCC', 2, 2)
('AC', 29, 29)
('ACA', 1, 2)
('ACBB', 1, 1)
('ACBC', 1, 1)
('ACC', 6, 6)
('ACCC', 6, 6)
('ACXCB', 1, 1)
('ACXX', 1, 1)
('AK', 1, 1)
('BA', 1, 1)
('BBBA', 1, 1)
('CA', 4, 4)
('CAAA', 1, 3)
('CAB', 1, 1)
('CABCC', 2, 2)
('CAC', 2, 2)
('CACC', 2, 2)
('CACCC', 1, 1)
('CCA', 1, 1)
('CCAC', 1, 1)
('KA', 1, 1)
('KABA', 1, 2)


In [86]:
sev_sum = 0 
for i,j in zip(qry.severity, qry.cnt):
    if 'A' in i:
        #print(i,j,j*i.count('A'))
        sev_sum+=j*i.count('A')
print(sev_sum)

696


In [ ]:
df.to_csv('example.csv',index=False)